In [ ]:
!pip install roboflow opencv-python-headless matplotlib requests tqdm

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="K0xg5GEEinqPgaqjKKzz")
project = rf.workspace("matyworkspace").project("damagedhealthytrafficsigns")
version = project.version(9)
dataset = version.download("yolov8")

In [ ]:
import os
import cv2
import glob

# Load image paths
image_dir = os.path.join(dataset.location, "test", "images")
label_dir = os.path.join(dataset.location, "test", "labels")
image_paths = glob.glob(os.path.join(image_dir, "*.jpg"))

# Load class names
class_names = ["damaged", "healthy"]

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance, ImageFilter
import os
from typing import Tuple, Optional

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt

def show_random_images_with_bboxes(image_paths, label_dir, class_names, N=9, cols=None):
    sample_paths = random.sample(image_paths, min(N, len(image_paths)))
    images = []
    bboxes = []
    labels = []

    for img_path in sample_paths:
        img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
        bbox_list = []
        label_list = []
        label_path = os.path.join(label_dir, os.path.basename(img_path).replace('.jpg', '.txt'))
        if os.path.exists(label_path):
            h_img, w_img = img.shape[:2]
            with open(label_path, "r") as f:
                for line in f:
                    class_id, cx, cy, w, h = map(float, line.strip().split())
                    x_center = cx * w_img
                    y_center = cy * h_img
                    width = w * w_img
                    height = h * h_img
                    x1 = int(x_center - width / 2)
                    y1 = int(y_center - height / 2)
                    x2 = int(x_center + width / 2)
                    y2 = int(y_center + height / 2)
                    bbox_list.append((x1, y1, x2, y2))
                    label_list.append(class_names[int(class_id)])
        images.append(img)
        bboxes.append(bbox_list)
        labels.append(label_list)

    cols = int(np.ceil(np.sqrt(N)) if cols is None else cols)
    rows = int(np.ceil(N / cols))

    fig, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 4))
    axes = axes.flatten()

    for i, (img, bbox_list, label_list) in enumerate(zip(images, bboxes, labels)):
        img_draw = img.copy()
        h_img, w_img = img.shape[:2]
        thickness = max(2, int(round(0.005 * (h_img + w_img) / 2)))
        font_scale = max(0.5, 0.002 * (h_img + w_img) / 2)
        for bbox, label in zip(bbox_list, label_list):
            x1, y1, x2, y2 = bbox
            color = (0, 255, 0) if label == "healthy" else (255, 0, 0)
            cv2.rectangle(img_draw, (x1, y1), (x2, y2), color, thickness)
            cv2.putText(img_draw, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, thickness)
        axes[i].imshow(img_draw)
        axes[i].axis('off')
    for j in range(i+1, len(axes)):
        axes[j].axis('off')
    plt.tight_layout()
    plt.show()

show_random_images_with_bboxes(image_paths, label_dir, class_names, N=8, cols=4)

In [ ]:
import requests
import io
import os

def send_image_for_prediction(image_path):
    with open(image_path, "rb") as f:
        image_data = f.read()
        files = {
            'image': ('image.jpg', io.BytesIO(image_data), 'image/jpeg')
        }
        prediction_url = os.environ.get('PREDICTION_URL', 'http://localhost:3000/predict')
        response = requests.post(
            prediction_url,
            files=files,
            timeout=30
        )
        return response

In [ ]:
import requests
import time
import numpy as np
import matplotlib.pyplot as plt

def draw_bboxes(img, bboxes, color=(0, 255, 0), labels=None, confidences=None):
    img = img.copy()
    for i, bbox in enumerate(bboxes):
        x1, y1, x2, y2 = int(bbox["x1"]), int(bbox["y1"]), int(bbox["x2"]), int(bbox["y2"])
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        label = ""
        if labels:
            label = labels[i]
        if confidences:
            label += f" {confidences[i]:.2f}"
        if label:
            cv2.putText(img, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return img

def compute_iou(boxA, boxB):
    xA = max(boxA["x1"], boxB["x1"])
    yA = max(boxA["y1"], boxB["y1"])
    xB = min(boxA["x2"], boxB["x2"])
    yB = min(boxA["y2"], boxB["y2"])

    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA["x2"] - boxA["x1"]) * (boxA["y2"] - boxA["y1"])
    boxBArea = (boxB["x2"] - boxB["x1"]) * (boxB["y2"] - boxB["y1"])

    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

In [ ]:
from tqdm import tqdm

IOU_THRESHOLD = 0.5

results = []
print(f"Processing {len(image_paths)} images...")

for image_path in tqdm(image_paths):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    filename = os.path.basename(image_path).replace('.jpg', '.txt')
    label_path = os.path.join(label_dir, filename)

    # Load ground truth bounding boxes
    gt_bboxes = []
    with open(label_path, "r") as f:
        for line in f:
            class_id, cx, cy, w, h = map(float, line.strip().split())
            h_img, w_img = image.shape[:2]
            x_center = cx * w_img
            y_center = cy * h_img
            width = w * w_img
            height = h * h_img
            x1 = int(x_center - width / 2)
            y1 = int(y_center - height / 2)
            x2 = int(x_center + width / 2)
            y2 = int(y_center + height / 2)
            gt_bboxes.append({
                "class": class_names[int(class_id)],
                "x1": x1, "y1": y1, "x2": x2, "y2": y2
            })

    predictions = send_image_for_prediction(image_path).json()
    predictions = sorted(predictions, key=lambda x: x['confidence'], reverse=True)

    pred_bboxes = [{**p['box'], **p} for p in predictions]


    gt_bboxes_predicted = [False for _ in gt_bboxes]
    for p in pred_bboxes:
        class_id = p['cls_score']

        # Find the closest ground truth bbox for label
        best_iou = 0
        actual_label = "unknown"
        gt_idx = None
        for i, gt in enumerate(gt_bboxes):
            if gt_bboxes_predicted[i]:
                continue
            iou = compute_iou(p, gt)
            if iou >= IOU_THRESHOLD and iou > best_iou:
                best_iou = iou
                actual_label = gt["class"]
                gt_idx = i
        if gt_idx is not None:
            gt_bboxes_predicted[gt_idx] = True

        # Show cropped image and label
        #plt.imshow(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))
        #plt.title(f"Actual: {actual_label} | Predicted: {predicted_label}")
        #plt.axis("off")
        #plt.show()

        # Save result
        results.append({
            "image": image_path,
            "actual_bbox": {
                "x1": int(gt_bboxes[gt_idx]["x1"]),
                "y1": int(gt_bboxes[gt_idx]["y1"]),
                "x2": int(gt_bboxes[gt_idx]["x2"]),
                "y2": int(gt_bboxes[gt_idx]["y2"])
            } if gt_idx is not None else {},
            "predicted_bbox": {
                "x1": int(p["x1"]),
                "y1": int(p["y1"]),
                "x2": int(p["x2"]),
                "y2": int(p["y2"])
            },
            "actual": actual_label,
            "score": class_id,
            "iou": best_iou,
            "confidence": p["confidence"]
        })
    for i, predicted in enumerate(gt_bboxes_predicted):
        if not predicted:
            results.append({
                "image": image_path,
                "actual_bbox": {
                    "x1": int(gt_bboxes[i]["x1"]),
                    "y1": int(gt_bboxes[i]["y1"]),
                    "x2": int(gt_bboxes[i]["x2"]),
                    "y2": int(gt_bboxes[i]["y2"])
                },
                "predicted_bbox": {},
                "actual": gt_bboxes[i]["class"],
                "score": 0,
                "iou": 0,
                "confidence": 0
            })
# Save results to JSON
with open("results.json", "w") as f:
    import json
    json.dump(results, f, indent=4)
        

In [ ]:
!pip install scikit-learn pandas

In [ ]:
import pandas as pd
import json
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [ ]:
with open("results.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [ ]:
healthy_confidence_threshold = 0.4
damaged_confidence_threshold = healthy_confidence_threshold

def map_actual(x):
    return 'background' if x == 'unknown' else x

def map_predicted(score, conf):
    healthy_conf = conf * score
    damaged_conf = conf * (1 - score)
    if healthy_conf >= healthy_confidence_threshold:
        if damaged_conf >= damaged_confidence_threshold:
            return 'healthy' if healthy_conf >= damaged_conf else 'damaged'
        return 'healthy'
    elif damaged_conf >= damaged_confidence_threshold:
        return 'damaged'
    return 'background'

df = pd.DataFrame(data)
df['actual_mapped'] = df['actual'].apply(map_actual)
df['predicted_mapped'] = df.apply(
    lambda row: map_predicted(row['score'], row['confidence']), axis=1
)

classes = ['healthy', 'damaged', 'background']

y_true = df['actual_mapped']
y_pred = df['predicted_mapped']

In [ ]:
import numpy as np

cm = confusion_matrix(y_true, y_pred, labels=classes)
cm_df = pd.DataFrame(cm, index=classes, columns=classes)


display(cm_df)

#cm[2, 2] = 0  # Set background-background to nan

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Only use 'healthy' and 'damaged' classes
labels = ['healthy', 'damaged']
precision = precision_score(df['actual_mapped'], df['predicted_mapped'], average=None, labels=labels)
recall = recall_score(df['actual_mapped'], df['predicted_mapped'], average=None, labels=labels)
f1 = f1_score(df['actual_mapped'], df['predicted_mapped'], average=None, labels=labels)

for cls, p, r, f in zip(labels, precision, recall, f1):
    print(f"{cls}: Precision={p:.3f}, Recall={r:.3f}, F1={f:.3f}")

In [ ]:
print(df.columns)

In [ ]:
from sklearn.metrics import average_precision_score

ious = np.arange(0.5, 1.0, 0.05)
aps = []

for iou_thr in ious:
    # Filter detections by IoU threshold
    df_iou = df.copy()

    # AP for healthy
    y_true_healthy = (df_iou['actual_mapped'] == 'healthy').astype(int)
    y_score_healthy = df_iou['confidence'] * df_iou['score'] * (df_iou['iou'] >= iou_thr).astype(float) * (df_iou['score'] >= 0.5).astype(float)
    ap_healthy = average_precision_score(y_true_healthy, y_score_healthy) if len(y_true_healthy) > 0 and y_true_healthy.sum() > 0 else np.nan

    # AP for damaged
    y_true_damaged = (df_iou['actual_mapped'] == 'damaged').astype(int)
    y_score_damaged = df_iou['confidence'] * (1 - df_iou['score']) * (df_iou['iou'] >= iou_thr).astype(float) * (df_iou['score'] <= 0.5).astype(float)
    ap_damaged = average_precision_score(y_true_damaged, y_score_damaged) if len(y_true_damaged) > 0 and y_true_damaged.sum() > 0 else np.nan

    aps.append({'iou': iou_thr, 'AP_healthy': ap_healthy, 'AP_damaged': ap_damaged})

# Convert to DataFrame for display
aps_df = pd.DataFrame(aps)
aps_df['mAP'] = aps_df[['AP_healthy', 'AP_damaged']].mean(axis=1)
display(aps_df)
map_50_95 = aps_df['mAP'].mean()

plt.figure(figsize=(8,5))
plt.plot(aps_df['iou'], aps_df['AP_healthy'], label='AP Healthy', marker='o', color='green')
plt.plot(aps_df['iou'], aps_df['AP_damaged'], label='AP Damaged', marker='o', color='red')
plt.plot(aps_df['iou'], aps_df['mAP'], label='mAP', marker='o', color='blue')
plt.axhline(y=map_50_95, color='purple', linestyle='--', alpha=0.7, label=f'mAP@0.50:0.95 = {map_50_95:.3f}')
plt.xlabel('IoU Threshold')
plt.ylabel('Average Precision (AP)')
plt.title('AP by class and mAP vs IoU Threshold')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


print(f"mAP@50:95 = {map_50_95:.3f}")

In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score

df_det = df.copy()
y_true_det = (df_det['actual_mapped'] != 'background').astype(int)
y_score_det = df_det['confidence'] 

precision_det, recall_det, thresholds = precision_recall_curve(y_true_det, y_score_det)
ap_det = average_precision_score(y_true_det, y_score_det)

plt.figure(figsize=(7, 5))
plt.plot(recall_det, precision_det, label=f"Detección (AP={ap_det:.2f})", color='blue')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve (Detección: objeto vs fondo)')
plt.legend()
#plt.ylim([0.0, 1.05])
#plt.xlim([0.0, 1.05])
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import f1_score

df_det = df.copy()

# Healthy
y_true_healthy = (df_det['actual_mapped'] == 'healthy').astype(int)
y_score_healthy = df_det['confidence'] * df_det['score'] * (df_det['score'] >= 0.5).astype(float)
precision_healthy, recall_healthy, thresholds_healthy = precision_recall_curve(y_true_healthy, y_score_healthy)
ap_healthy = average_precision_score(y_true_healthy, y_score_healthy)

# Damaged
y_true_damaged = (df_det['actual_mapped'] == 'damaged').astype(int)
y_score_damaged = df_det['confidence'] * (1 - df_det['score']) * (df_det['score'] <= 0.5).astype(float)
precision_damaged, recall_damaged, thresholds_damaged = precision_recall_curve(y_true_damaged, y_score_damaged)
ap_damaged = average_precision_score(y_true_damaged, y_score_damaged)

plt.figure(figsize=(7, 5))
plt.plot(recall_healthy, precision_healthy, label=f"Healthy (AP={ap_healthy:.2f})", color='green')
plt.plot(recall_damaged, precision_damaged, label=f"Damaged (AP={ap_damaged:.2f})", color='red')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve by class')
plt.legend()
plt.xticks(np.arange(0, 1.01, 0.1))
plt.yticks(np.arange(0, 1.01, 0.1))
plt.xlim([0.0, 1.02])
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure(figsize=(7, 5))
plt.plot(thresholds_healthy, precision_healthy[1:], label=f"Precision Healthy", color='limegreen')
plt.plot(thresholds_healthy, recall_healthy[1:], label=f"Recall Healthy", color='darkgreen')
plt.plot(thresholds_damaged, precision_damaged[1:], label=f"Precision Damaged", color='red')
plt.plot(thresholds_damaged, recall_damaged[1:], label=f"Recall Damaged", color='darkred')

# Add horizontal dotted lines for max damage recall and 95% of max
max_damage_recall = np.max(recall_damaged[1:])
recall_95_percent = 0.964 * max_damage_recall
plt.axhline(y=max_damage_recall, color='darkred', linestyle='--', alpha=0.7, label=f'Max Damage Recall ({max_damage_recall:.3f})')
plt.axhline(y=recall_95_percent, color='darkred', linestyle=':', alpha=0.7, label=f'>95% Max Damage Recall ({recall_95_percent:.3f})')
#plt.axvline(x=0.4, color='purple', linestyle='--', alpha=0.7, label='Selected Threshold (0.4)')
plt.xlabel('Combined Score Threshold')
plt.ylabel('Precision/Recall')
plt.title('Precision and Recall vs Combined Score Threshold by class')
plt.legend()
plt.xticks(np.arange(0, 1.01, 0.1))
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.02])
plt.yticks(np.arange(0, 1.01, 0.1))
plt.grid(True)
plt.tight_layout()
plt.show()


# Compute F1-score for each threshold for healthy
f1_healthy = []
for thr in thresholds_healthy:
    preds = (y_score_healthy >= thr).astype(int)
    f1 = f1_score(y_true_healthy, preds)
    f1_healthy.append(f1)

# Compute F1-score for each threshold for damaged
f1_damaged = []
for thr in thresholds_damaged:
    preds = (y_score_damaged >= thr).astype(int)
    f1 = f1_score(y_true_damaged, preds)
    f1_damaged.append(f1)

plt.figure(figsize=(7, 5))
plt.plot(thresholds_healthy, f1_healthy, label="F1 Healthy", color='green')
plt.plot(thresholds_damaged, f1_damaged, label="F1 Damaged", color='red')
plt.axvline(x=0.4, color='purple', linestyle='--', alpha=0.7, label='Selected Threshold (0.4)')
plt.xlabel('Combined Score Threshold')
plt.ylabel('F1-score')
plt.title('F1-score vs Combined Score Threshold by class')
plt.legend()
plt.xticks(np.arange(0, 1.01, 0.1))
plt.yticks(np.arange(0, 1.01, 0.1))
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.02])
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import random
import cv2

import matplotlib.pyplot as plt

def draw_all_predictions_on_image(img_path, df):
    img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    sample_df = df[df['image'] == img_path]
    img_draw = img.copy()
    h_img, w_img = img.shape[:2]
    thickness = max(2, int(round(0.005 * (h_img + w_img) / 2)))
    font_scale = max(0.5, 0.002 * (h_img + w_img) / 2)
    for _, row in sample_df.iterrows():
        bbox = row['predicted_bbox'] if row['predicted_bbox'] else row['actual_bbox']
        
        if bbox and all(k in bbox for k in ['x1', 'y1', 'x2', 'y2']):
            x1, y1, x2, y2 = bbox['x1'], bbox['y1'], bbox['x2'], bbox['y2']
            truth = row['actual_mapped']
            label = row['predicted_mapped']

            if truth == label:
                color = (0, 255, 0) if label != 'background' else (128, 128, 0)
            elif label == 'background':
                color = (255, 165, 0)
            else:
                color = (255, 0, 0)
            cv2.rectangle(img_draw, (x1, y1), (x2, y2), color, thickness)
            cv2.putText(img_draw, label, (x1, y1 + int(font_scale*13)), cv2.FONT_HERSHEY_SIMPLEX, font_scale*0.6, color, int(thickness))
    return img_draw

# Pick random sample of images
N = 20
sample_images = random.sample(image_paths, N)
for img_path in sample_images:
    annotated_img = draw_all_predictions_on_image(img_path, df)
    plt.figure(figsize=(8, 8))
    plt.imshow(annotated_img)
    plt.axis('off')
    plt.show()

In [ ]:
import cv2
import numpy as np
import random
import os

import matplotlib.pyplot as plt

def draw_bbox(img, bbox, label, color, thickness=2, font_scale=0.7):
    x1, y1, x2, y2 = bbox
    img = img.copy()
    cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness)
    cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, thickness)
    return img

def get_bbox_tuple(bbox_dict):
    if bbox_dict and all(k in bbox_dict for k in ['x1', 'y1', 'x2', 'y2']):
        return (bbox_dict['x1'], bbox_dict['y1'], bbox_dict['x2'], bbox_dict['y2'])
    return None

def crop_image(img, bbox, pad):
    if bbox is None:
        return img
    x1, y1, x2, y2 = bbox
    h, w = img.shape[:2]
    x1 = max(0, x1 - pad)
    y1 = max(0, y1 - pad)
    x2 = min(w, x2 + pad)
    y2 = min(h, y2 + pad)
    return img[y1:y2, x1:x2]

def show_examples_grid(df, N=12, cols=4):
    samples = list(df.index)[:N]

    rows = int(np.ceil(N / cols))
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 5))
    axes = axes.flatten()

    for i, idx in enumerate(samples):
        row = df.loc[idx]
        img_path = row['image']
        img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)

        gt_bbox = get_bbox_tuple(row['actual_bbox'])
        pred_bbox = get_bbox_tuple(row['predicted_bbox'])

        # Crop around the predicted bbox if available, else ground truth bbox, else show full image
        crop_bbox = pred_bbox if pred_bbox else gt_bbox
        pad = 50
        img_crop = crop_image(img, crop_bbox, pad=pad)

        img_draw = img_crop.copy()
        # Draw ground truth annotation
        if gt_bbox:
            # Adjust bbox coordinates for crop
            if crop_bbox:
                x1, y1, x2, y2 = gt_bbox
                cx1, cy1, _, _ = crop_bbox
                adj_bbox = (max(0, x1-cx1+pad), max(0, y1-cy1+pad), max(0, x2-cx1+pad), max(0, y2-cy1+pad))
            else:
                adj_bbox = gt_bbox
            img_draw = draw_bbox(img_draw, adj_bbox, f"GT: {row['actual_mapped']}", color=(0, 255, 0), thickness=3)
        # Draw prediction annotation
        if pred_bbox:
            if crop_bbox:
                x1, y1, x2, y2 = pred_bbox
                cx1, cy1, _, _ = crop_bbox
                adj_bbox = (max(0, x1-cx1+pad), max(0, y1-cy1+pad), max(0, x2-cx1+pad), max(0, y2-cy1+pad))
            else:
                adj_bbox = pred_bbox
            img_draw = draw_bbox(img_draw, adj_bbox, f"Pred: {row['predicted_mapped']}", color=(255, 0, 0), thickness=2)

        axes[i].imshow(img_draw)
        #axes[i].set_title(title)
        axes[i].axis('off')

    for j in range(i+1, len(axes)):
        axes[j].axis('off')
    plt.tight_layout()
    plt.show()

# Usage: show grid of FP, FN, TN examples with annotations
print("False Positives (FP):")
show_examples_grid(df[(df['actual_mapped'] == 'background') & (df['predicted_mapped'] != 'background')], N=30, cols=5)
print("False Backgrounds (FN):")
show_examples_grid(df[(df['actual_mapped'] != 'background') & (df['predicted_mapped'] == 'background')], N=75, cols=5)
print("Misclassifications (FN):")
show_examples_grid(df[(df['actual_mapped'] != 'background') & (df['predicted_mapped'] != 'background') & (df['actual_mapped'] != df['predicted_mapped'])], N=75, cols=5)
print("True Negatives (TN):")
show_examples_grid(df[(df['actual_mapped'] == 'background') & (df['predicted_mapped'] == 'background')], N=40, cols=5)
print("True Positives (TP):")
show_examples_grid(df[(df['actual_mapped'] != 'background') & (df['predicted_mapped'] != 'background') & (df['actual_mapped'] == df['predicted_mapped'])], N=75, cols=5)